In [17]:
#from google.colab import drive
#drive.mount('/content/drive')
from convert import *
from function1 import *
import numpy as np

film_sayisi = 3
window_size = 5

films = list(range(film_sayisi))

# Loading the dictionary
with open("dictionary.txt","r",encoding="utf8") as file:
    dictionary = [string.replace("\n","") for string in file.readlines()]

In [18]:
from keras.models import load_model
from keras.optimizers import Adagrad
model = load_model("models/Dense_50epochs.h5",compile=False)
opt = Adagrad(learning_rate = 0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Creating scripts[][]
scripts = []

for j in films:
    with open(f"{j}_enc.txt","r") as file:
        scripts.append(list(map(int,list(file.read().splitlines()))))

# Turning int to [int]
def func(liste):
    return [[x] for x in liste]
scripts = list(map(func,scripts))

# To categorical -> [0,0,0,...,1,...0,0,0]
from keras.utils import to_categorical
for j in films:
    scripts[j] = to_categorical(scripts[j],num_classes=len(dictionary)+1,dtype="byte")

# Creating inputs and outputs lists
inputs = []
outputs = []
for j in films:
    inputs.append(None)
    outputs.append(None)

# Creating sublists
for j in films:
    inputs[j],outputs[j] = create_sublists(scripts[j],window_size)

# Removing [films] layer in lists
inputs_temp = []
for j in films:
    for k in range(len(inputs[j])):
        inputs_temp.append(inputs[j][k])
inputs = inputs_temp
del inputs_temp
outputs_temp = []
for j in films:
    for k in range(len(outputs[j])):
        outputs_temp.append(outputs[j][k])
outputs = outputs_temp
del outputs_temp

# Creating train data and test data, x is for input and y is for output
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3, random_state=42)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
# Training the model

model.fit(x_test, y_test, epochs=20, batch_size=16)

Epoch 1/20
1043/1043 [==============================] - 67s 65ms/step - loss: 6.0126 - accuracy: 0.1330
Epoch 2/20
1043/1043 [==============================] - 67s 64ms/step - loss: 5.7705 - accuracy: 0.1556
Epoch 3/20
1043/1043 [==============================] - 67s 64ms/step - loss: 5.5697 - accuracy: 0.1773
Epoch 4/20
1043/1043 [==============================] - 67s 64ms/step - loss: 5.3949 - accuracy: 0.1963
Epoch 5/20
1043/1043 [==============================] - 67s 64ms/step - loss: 5.2390 - accuracy: 0.2144
Epoch 6/20
1043/1043 [==============================] - 68s 65ms/step - loss: 5.0971 - accuracy: 0.2324
Epoch 7/20
1043/1043 [==============================] - 67s 65ms/step - loss: 4.9679 - accuracy: 0.2506
Epoch 8/20
1043/1043 [==============================] - 67s 65ms/step - loss: 4.8465 - accuracy: 0.2678
Epoch 9/20
1043/1043 [==============================] - 67s 65ms/step - loss: 4.7342 - accuracy: 0.2829
Epoch 10/20
1043/1043 [==============================] - 67s 64m

In [ ]:
# Saving the model
model.save("drive/MyDrive/Colab Notebooks/Dense_70epochs.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 36690)             0         
                                                                 
 dense (Dense)               (None, 7338)              269238558 
                                                                 
 dense_1 (Dense)             (None, 7338)              53853582  
                                                                 
Total params: 323092140 (1.20 GB)
Trainable params: 323092140 (1.20 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Testing the accuracy of model on both datasets
pred_output = model.predict(x_train)
pred_output2 = model.predict(x_test)

522/522 [==============================] - 62s 118ms/step


In [23]:
# Testing the model on train data
length = len(y_train)
counter = 0
for i in range(length):
    a = toInt(pred_output[i])
    b = y_train[i]
    if a.tolist() == b.tolist():
        counter+=1

accuracy = counter * 100 / length
print(f"Accuracy: %{accuracy}")

# Testing the model on test data
length = len(y_test)
counter = 0
for i in range(length):
    a = toInt(pred_output2[i])
    b = y_test[i]
    if a.tolist() == b.tolist():
        counter+=1

accuracy = counter * 100 / length
print(f"Accuracy: %{accuracy}")

Accuracy: %37.58546239654552
Accuracy: %43.25637181409295
